In [15]:
# !pip install beautifulsoup4

In [16]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup

# for문으로 크롤링 

In [ ]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller

chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

In [ ]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [ ]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [108]:
BASE_URL = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sort=2&photo=0&field=0&pd=3&"
# https://finance.naver.com/sise/sise_market_sum.naver?&page=1
start = '1990.01.01'
end = '2005.05.01'
search_span= "ds=" + start + "&de=" + end + "&a:all&start=" # 기본값


In [86]:
BASE_URL = BASE_URL + search_span

service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(BASE_URL+"1")

# soup = BeautifulSoup(BASE_URL, "html.parser")
# soup.find_all('btn',{'aria-pressed':"false"})
last_num = driver.find_element('xpath', '//*[@id="main_pack"]/div[2]/div/div/a[2]').text
# //*[@id="main_pack"]/div[2]/div/div/a[8]
last_num = int(last_num)
last_num

2

In [109]:
# 뉴스 데이터 수집
news_list = []
date_list = []
title_list = []

BASE_URL = BASE_URL + search_span

service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(BASE_URL+"1")


for i in range(500):  
    
    # 언론사
    newses = driver.find_elements('css selector', 'a.info.press')
    temp_list = []
    for news in newses:
        temp_list.append(news.text)
#         temp_list = [x for x in temp_list ]
    news_list = news_list + temp_list 
    
    # 날짜
    dates = driver.find_elements('css selector', 'span.info')
    temp_list = []
    for date in dates:
        temp_list.append(date.text)
#         temp_list = [x for x in temp_list ]
    date_list = date_list + temp_list 
    
    # 제목
    titles = driver.find_elements('css selector', 'a.news_tit')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
#         temp_list = [x for x in temp_list ]
    title_list = title_list + temp_list 
    last_num = int(driver.find_element('xpath', '//*[@id="main_pack"]/div[2]/div/div/a'.format(i+1)).text) 
    # 문제점 페이지 수 가져오려니 1, 2 이렇게 나옴
    print(last_num)
#     print(i)
    
    # 다음페이지 클릭, next 비활성화 되면 멈추는 경우 2가지
    if driver.find_element('css selector', 'a.btn_next').get_attribute('aria-disabled') == 'false':
        driver.find_element('css selector', 'a.btn_next').click()
        continue;
    # 다음 페이지가 비활성화 됐을 때
    # 1. page 400일 대 (날짜 업데이트)
    elif driver.find_element('css selector', 'a.btn_next').get_attribute('aria-disabled') == 'true':
        # 날짜를 마지막 뉴스 날짜로 업데이트
        if last_num == 400:
            search_span = "ds=" + date_list[-1] + "&de=" + end + "&a:all&start=" # 시작, 끝 날짜 설정
            BASE_URL = BASE_URL + search_span
            service = ChromeService(executable_path=ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service)
            driver.get(BASE_URL+"1")
            continue;

        # 2. 마지막 page가 400보다 작을 때
        elif last_num < 400:
            break;

    # 검색 결과가 없을 때
    else: # 검색결과 없음 
         break;
        
    time.sleep(0.001)
    
print(len(title_list))
# print(news_list)
print(date_list)
# print(title_list)


1
2
3
4
5
7
9
11
13
15


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[2]/div/div/a[11]"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E85FD3+2187219]
	Ordinal0 [0x00E1E6D1+1763025]
	Ordinal0 [0x00D33E78+802424]
	Ordinal0 [0x00D61C10+990224]
	Ordinal0 [0x00D61EAB+990891]
	Ordinal0 [0x00D8EC92+1174674]
	Ordinal0 [0x00D7CBD4+1100756]
	Ordinal0 [0x00D8CFC2+1167298]
	Ordinal0 [0x00D7C9A6+1100198]
	Ordinal0 [0x00D56F80+946048]
	Ordinal0 [0x00D57E76+949878]
	GetHandleVerifier [0x011290C2+2721218]
	GetHandleVerifier [0x0111AAF0+2662384]
	GetHandleVerifier [0x00F1137A+526458]
	GetHandleVerifier [0x00F10416+522518]
	Ordinal0 [0x00E24EAB+1789611]
	Ordinal0 [0x00E297A8+1808296]
	Ordinal0 [0x00E29895+1808533]
	Ordinal0 [0x00E326C1+1844929]
	BaseThreadInitThunk [0x7534FA29+25]
	RtlGetAppContainerNamedObjectPath [0x772A7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x772A7A6E+238]


In [102]:
date_list

['1990.01.17.',
 '1990.01.17.',
 '1990.01.17.',
 '1990.01.19.',
 '1990.01.21.',
 '1990.01.21.',
 '1990.01.23.',
 '1990.01.24.',
 '1990.01.24.',
 '1990.01.24.',
 '1990.01.24.',
 '1990.01.25.',
 '1990.02.01.',
 '1990.02.02.',
 '1990.02.02.',
 '1990.02.02.',
 '1990.02.18.',
 '1990.02.19.',
 '1990.02.19.',
 '1990.02.20.',
 '1990.02.20.',
 '1990.02.20.',
 '1990.02.21.',
 '1990.02.22.',
 '1990.02.22.',
 '1990.02.23.',
 '1990.02.23.',
 '1990.02.24.',
 '1990.02.26.',
 '1990.02.26.',
 '1990.03.02.',
 '1990.03.02.',
 '1990.03.02.',
 '1990.03.03.',
 '1990.03.06.',
 '1990.03.06.',
 '1990.03.06.',
 '1990.03.06.',
 '1990.03.06.',
 '1990.03.07.',
 '1990.03.07.',
 '1990.03.07.',
 '1990.03.09.',
 '1990.03.10.',
 '1990.03.10.',
 '1990.03.12.',
 '1990.03.12.',
 '1990.03.12.',
 '1990.03.13.',
 '1990.03.14.',
 '1990.03.14.',
 '1990.03.17.',
 '1990.03.17.',
 '1990.03.17.',
 '1990.03.17.',
 '1990.03.17.',
 '1990.03.21.',
 '1990.03.21.',
 '1990.03.21.',
 '1990.03.22.',
 '1990.03.22.',
 '1990.03.22.',
 '1990.0

In [ ]:
x = "ds=" + date_list[-1] + "&de=1990.02.01&a:all&start="
type(x)

In [103]:
# df로 만들기

df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])

df

# csv로 뽑기
# df.to_csv('.csv')

,언론사,날짜,제목
0,연합뉴스,1990.01.17.,朴守敎-林正明 코트서 벤치 대결로
1,연합뉴스,1990.01.17.,"EC, 한국산 CDP 반덤핑 확정관세 결정"
2,연합뉴스,1990.01.17.,범용 CAD 개발
3,연합뉴스,1990.01.19.,"광고업계,제조업 불황속 호황 누려"
4,연합뉴스,1990.01.21.,-농구대잔치-
...,...,...,...
104,연합뉴스,1990.04.24.,유형별 위반사례
105,연합뉴스,1990.04.24.,8개 財閥기업 不動産취득 규정 위반
106,연합뉴스,1990.04.24.,별 표
107,연합뉴스,1990.04.24.,<표> 유형별 위반사례


In [ ]:
for i in range(5):
    print(i)